In [1]:
from demo_track import Predictor
import torch
from torch2trt import TRTModule
# import sys
# import os
# sys.path.append('/home/leejaehyeok/trackingProject/ocsort/OC_SORT/')

/home/leejaehyeok/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trt_file = '/home/leejaehyeok/trackingProject/ocsort/OC_SORT/YOLOX_outputs/yolox_dancetrack_test/model_trt.pth'

In [3]:

            
x = torch.ones((1, 3, 800, 1440), device="cuda")

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(trt_file))


[07/12/2022-10:28:25] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.


<All keys matched successfully>

In [5]:
with torch.no_grad():
    outputs = model_trt(x)

[07/12/2022-10:28:39] [TRT] [W] The enqueue() method has been deprecated when used with engines built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. Please use enqueueV2() instead.
[07/12/2022-10:28:39] [TRT] [W] Also, the batchSize argument passed into this function has no effect on changing the input shapes. Please use setBindingDimensions() function to change input shapes instead.


: 

In [7]:
def decode_outputs(outputs, dtype):
    grids = []
    strides = []
    hw = [[100,180],[50,90],[25,45]]
    cls_strides = [8, 16, 32]
    for (hsize, wsize), stride in zip(hw, cls_strides):
        yv, xv = torch.meshgrid([torch.arange(hsize), torch.arange(wsize)])
        a = torch.stack((xv,yv))
        print(a.shape)
        grid = torch.stack((xv, yv), 2).view(1, -1, 2)
        grids.append(grid)
        shape = grid.shape[:2]
        strides.append(torch.full((*shape, 1), stride))

    grids = torch.cat(grids, dim=1).type(dtype)
    strides = torch.cat(strides, dim=1).type(dtype)

    outputs[..., :2] = (outputs[..., :2] + grids) * strides
    outputs[..., 2:4] = torch.exp(outputs[..., 2:4]) * strides
    return outputs

In [8]:
outputs = decode_outputs(outputs, dtype=outputs.type())

torch.Size([2, 100, 180])
torch.Size([2, 50, 90])
torch.Size([2, 25, 45])
